In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sentence_transformers import CrossEncoder
import random

In [5]:
df = pd.read_json("../data/intent_data.json")
df.head()

,intent_id,intent_name,main_listening_function,listening_functions,listening_function_factors,survey_intent_names
0,0,(Self-) Reflection / Introspection,Because it helps me think about myself.,"[Because it helps me think about myself., Beca...","[1.0, 0.7000000000000001, 0.4, 0.5, 0.30000000...","[reflection, Self thought, Speaking through mu..."
1,1,Nostalgia,Because it helps me reminisce.,"[Because it helps me reminisce., Because it re...","[1.0, 0.9, 0.1, 0.6000000000000001]","[nostalgia, reminiscence , de-stress, Nostalgi..."
5,2,Relaxing,Because it calms me.,"[Because it is therapy for my soul., Because i...","[0.6000000000000001, 0.8, 0.30000000000000004,...","[Soothing, Peace, companion, Safety, Relief, R..."
3,3,Meaningful Engagement / Purpose,Because it adds meaning to my life.,[Because it gives me something that is mine al...,"[0.4, 1.0, 0.6000000000000001, 0.1, 0.2]","[Heartfelt, purpose, Concentrating, Music frie..."
7,4,Motivation,Because it prevents me from being bored while ...,[Because I need it in the background while I d...,"[0.6000000000000001, 1.0, 0.7000000000000001, ...","[Distraction , Boredom prevention , Flying tim..."


In [10]:
intent_id_to_name = {i: n for i, n in zip(df['intent_id'], df['intent_name'])}

## Load models
First, the models used for comparision are loaded. You can add your own models here.
all_is_st defines if it is a sentence transformer or not, which is then further used to compute the right code.

In [17]:
m_names = ['all-MiniLM-L12-v2', 'all-mpnet-base-v2', 'quora-distilbert-base', 'stsb-roberta-base', 'ensemble']
models = [SentenceTransformer(m_names[0]), SentenceTransformer(m_names[1]), SentenceTransformer(m_names[2]), CrossEncoder("cross-encoder/" + m_names[3]), None]
all_is_st = [True, True, True, False, False]

## Get Intent Vector

An intent vector consists of 32 values. A similarity score to each intent cluster. An intent cluster consists of multiple music listening functions. The similarity is computed to all music listening functions and are then averaged over all music listening functions.
The clusters were evaluated through a user study, from which a factor for each music listening function is determined, which shows how fitting a music listening function is for an intent cluster (listenin_functions_factors).
This factors can be used further for scaling the similarity scores to the music listening functions.

In [43]:
def get_intent_vec(text, model, sentence_transformer=False, with_score=False):
    intent_vec = np.zeros((32,))

    if sentence_transformer:
        intent_embs = []
        for q in df['listening_functions']:
            intent_embs.append(model.encode(q, normalize_embeddings=True))
        
        emb = model.encode(text, normalize_embeddings=True)
        
        for i, i_emb, scores in zip(df['intent_id'], intent_embs, df['listening_function_factors']):
            sim = cosine_similarity(emb.reshape(1, -1), i_emb)
            if with_score:
                intent_vec[i] = (scores * sim).mean()
            else:
                intent_vec[i] = (sim).mean()
    else:
        for i, qs, scores in zip(df['intent_id'], df['listening_functions'], df['listening_function_factors']):
            sim = model.predict([(text, q) for q in qs])
            if with_score:
                intent_vec[i] = (scores * sim).mean()
            else:
                intent_vec[i] = (sim).mean()

    intent_vec = intent_vec / (np.abs(intent_vec).sum() + 1)#normalize(intent_vec.reshape(1, -1))
    
    return intent_vec

## Analysis of Playlist title

In this section, the intent vector for a playlist title t is computed over all models loaded previously above. The computation differs if it is a Sentence Transformer, Cross Encoder or Ensemble. Make sure to specifiy it correctly, if you add another model. The intent vector can be computed in two ways: with_score = True or False.
with_score defines if the similarity score is scaled by the factor computed through a user study, which determined on how fitting a music listening function is for a specific intent (cluster). With this factor, the simiarity to a music listening function is scaled between 0 - 1.0.

In [41]:
t = "relax and take it easy"

In [42]:
m_name_to_intent_vec = {}
print("Not scored")
print("---")

for m_name, model, is_st in zip(m_names, models, all_is_st):
    if m_name == "ensemble":
        i_vecs = []

        for m, (i_vec, is_st) in m_name_to_intent_vec.items():
            if is_st:
                i_vecs.append(i_vec)

        m_name_to_intent_vec[m_name] = (np.array(i_vecs).mean(axis=0), is_st)
        print(m_name, "has identified the intent:", intent_id_to_name[m_name_to_intent_vec[m_name][0].argmax()])
    else: 
        m_name_to_intent_vec[m_name] = (get_intent_vec(t, model, sentence_transformer=is_st, with_score=False), is_st)
        print(m_name, "has identified the intent:", intent_id_to_name[m_name_to_intent_vec[m_name][0].argmax()])

Not scored
---
all-MiniLM-L12-v2 has identified the intent: Relief
all-mpnet-base-v2 has identified the intent: Escape
quora-distilbert-base has identified the intent: (Self-) Reflection / Introspection
stsb-roberta-base has identified the intent: Relief
ensemble has identified the intent: Escape


In [19]:
m_name_to_intent_vec = {}
print("Scored")
print("---")

for m_name, model, is_st in zip(m_names, models, all_is_st):
    if m_name == "ensemble":
        i_vecs = []

        for m, (i_vec, is_st) in m_name_to_intent_vec.items():
            if is_st:
                i_vecs.append(i_vec)

        m_name_to_intent_vec[m_name] = (np.array(i_vecs).mean(axis=0), is_st)
        print(m_name, "has identified the intent:", intent_id_to_name[m_name_to_intent_vec[m_name][0].argmax()])
    else: 
        m_name_to_intent_vec[m_name] = (get_intent_vec(t, model, sentence_transformer=is_st, with_score=True), is_st)
        print(m_name, "has identified the intent:", intent_id_to_name[m_name_to_intent_vec[m_name][0].argmax()])

Scored
---
all-MiniLM-L12-v2 has identified the intent: Mood Enhancement
all-mpnet-base-v2 has identified the intent: Escape
quora-distilbert-base has identified the intent: Mood Enhancement
stsb-roberta-base has identified the intent: Relief
ensemble has identified the intent: Mood Enhancement


## Analysis of intents

In this section, we are exploring the playlist titles highest ranked for each intent. First, just one model is selected. This model is used to create the intent vector for 100 random playlist titles (or how many you want). Then, for each intent, the highest top k (in this case 3) playlist titles are shown.

In [20]:
df_titles = pd.read_json("../dashboard/data/playlist_data_scored.json")
df_titles.head()

,playlist,x,y,intent_vec,intent,score,model
0,house 2016,14.109623,4.595580,"[0.0266262185, 0.0339518588, 0.0138493096, 0.0...",15,0.038960,stsb-roberta-base
1,recently saved,13.295147,4.221951,"[0.0348517086, 0.0358242825, 0.030349031300000...",11,0.054904,stsb-roberta-base
2,smiley,12.932446,5.451835,"[0.0268185443, 0.0314533076, 0.0236492272, 0.0...",14,0.055233,stsb-roberta-base
3,edu,13.225877,6.014919,"[0.024570571200000002, 0.0209092298, 0.0194485...",7,0.038853,stsb-roberta-base
4,luv,12.855105,5.924630,"[0.0243315466, 0.0264571743, 0.0252842036, 0.0...",14,0.044520,stsb-roberta-base


In [38]:
idx = 1

m_name = m_names[idx]
model = models[idx]
is_st = all_is_st[idx]
with_score = True

In [39]:
title_to_intent_vec = {}

random.seed(0)
titles = random.choices(list(df_titles['playlist']), k=100)

for t in tqdm(titles):
    if m_name == "ensemble":
        i_vecs = []

        for name, m, m_is_st in zip(m_names, models, all_is_st):
            if m_is_st:
                i_vecs.append(get_intent_vec(t, m, sentence_transformer=m_is_st, with_score=with_score))

        title_to_intent_vec[t] = np.array(i_vecs).mean(axis=0)
    else: 
        title_to_intent_vec[t] = get_intent_vec(t, model, sentence_transformer=is_st, with_score=with_score)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:02<00:00,  3.63s/it]


In [40]:
titles = list(title_to_intent_vec.keys())
intent_vecs = np.array(list(title_to_intent_vec.values()))

for i in range(32):
    print(intent_id_to_name[i])
    i_vec_values = (intent_vecs[:, i] - intent_vecs[:, i].min()) / (intent_vecs[:, i].max() - intent_vecs[:, i].min())

    for idx in (-i_vec_values).argsort()[:3]:
        print("-", titles[idx], i_vec_values[idx])
    print("---")

(Self-) Reflection / Introspection
- psyche 1.0
- melancholia 0.9037532582902956
- reflective 0.8917293231464406
---
Nostalgia
- throwback music 1.0
- old rb 0.8174198350339433
- sufjan stevens – carrie  lowell 0.8110021575565418
---
Relaxing
- lay back 1.0
- breathe 0.9417921444773819
- welcome home 0.8536102081073523
---
Meaningful Engagement / Purpose
- pura vida 1.0
- let go 0.9414262192217849
- happy hardcore 0.9196462368602213
---
Motivation
- do work 1.0
- music3 0.8583540317422206
- ambiente 0.7955564929640777
---
Distraction
- melancholia 1.0
- breathe 0.9118532089423605
- ambiente 0.8983539979192346
---
Emotion (regulation)
- boys dont cry 1.0
- let go 0.868373919460224
- feels like summer 0.8416941311952069
---
Support
- milo 1.0
- trend 0.8966264873907663
- reflective 0.8365962663122087
---
Calming
- happy hardcore 1.0
- let go 0.9990842694224411
- melancholia 0.9756960273462356
---
Creativity
- ambiente 1.0
- psyche 0.9589098558898743
- reflective 0.9444704777674695
---
Le